# GSM8K

In [1]:
import json


def load_jsonl(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [2]:
reg_test = load_jsonl('./gsm8k_test_responses.json')
symb_test = load_jsonl('./gsm8k_test_responses_sym.json')

In [3]:
import re


def has_generated_correct_final_ans(response, gt):
    last_step_gen = response.split('\n')[-1]
    last_step_sol = gt.split('\n')[-1]
    
    try:
        final_solution = float(re.findall("the answer is: ([-+]?\d*\.?\d+)", last_step_sol.lower())[0])
        gen_solution = float(re.findall("the answer is: ([-+]?\d*\.?\d+)", last_step_gen.lower())[0])
        
        if final_solution == gen_solution:
            return True
    except Exception as e:
        print("couldn't parse answer")
    return False


for x in reg_test:
    x['correct'] = has_generated_correct_final_ans(x['generated_solution'], x['solution'])
    
for x in symb_test:
    x['correct'] = has_generated_correct_final_ans(x['generated_solution'], x['solution'])
    
    

couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't parse answer
couldn't p

In [4]:
reg_acc = len([x for x in reg_test if x['correct']]) / len(reg_test)
symb_acc = len([x for x in symb_test if x['correct']]) / len(symb_test)

reg_acc, symb_acc

(0.7488584474885844, 0.7427701674277016)

In [8]:
[x for x in symb_test if x['id'] == 714]

[{'id': 714,
  'problem': 'James decides to pick some blueberries.  It cost $20 to go picking and then another $1.5 per pound.  He picked 30 pounds of blueberries.  How much did he save compared to buying at the store for $2.5 a pound?',
  'solution': 'He paid 30*1.5=$<<30*1.5=45>>45 for the blueberries he picked\nSo after the entrance fee it was 45+20=$<<45+20=65>>65\nTo buy them at the grocery store it would cost 30*2.5=$<<30*2.5=75>>75\nSo he saved 75-65=$<<75-65=10>>10\nThe answer is: 10',
  'generated_solution': 'It costs James 1.5*30=$<<1.5*30=45.0>>45.0 to pick the blueberries.\nThe cost at the store for 30 pounds would be 2.5*30=$<<2.5*30=75.0>>75.0\nSo he saved 75-45=$<<75-45=30>>30 by picking his own blueberries\nThe answer is: 30<|eot_id|>',
  'correct': False}]

In [7]:
# saving examples that it couldn't answer

incorrects = [x for x in reg_test if not x['correct']]
print("total incorrect examples:", len(incorrects))
incorrects

total incorrect examples: 330


[{'id': 7,
  'problem': 'Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the beginning. How load does it take to download the file?',
  'solution': 'First find how many gigabytes are in 40% of the file: 200 GB * 40% = <<200*40*.01=80>>80 GB\nThen divide that number by the download rate to find the time until Windows restarts: 80 GB / 2 GB/minute = <<80/2=40>>40 minutes\nThen find the time to download the whole file after the restart: 200 GB / 2 GB/minute = <<200/2=100>>100 minutes\nThen add the time to download 40% of the file, to download the whole file, and to wait for Windows to update: 40 minutes + 100 minutes + 20 minutes = <<40+100+20=160>>160 minutes\nThe answer is: 160',
  'generated_solution': "40% of 200 GB is 0.4 * 200 = 80 GB.\nThis means Carla has to download the first 80 GB at 2 GB/minute, whic

In [11]:
with open('hard_test.json', 'w') as f:
    incorrects = [{k:v for k,v in x.items() if k in ['id', 'problem', 'solution']} for x in incorrects]
    json.dump(incorrects, f)

# MATH

## load and preprocess MATH from HF

In [35]:
import datasets

ds = datasets.load_dataset("lighteval/MATH", 'all')
ds

DatasetDict({
    train: Dataset({
        features: ['problem', 'level', 'type', 'solution'],
        num_rows: 7500
    })
    test: Dataset({
        features: ['problem', 'level', 'type', 'solution'],
        num_rows: 5000
    })
})

In [47]:

def convert_to_list_of_dict(ds):
    list_of_dict = []
    for d in ds:
        list_of_dict.append(d)
    return list_of_dict

convert_to_list_of_dict(ds['test'])[:3]

[{'problem': 'How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?',
  'level': 'Level 3',
  'type': 'Algebra',
  'solution': 'The denominator of the rational function factors into $x^2+x-6=(x-2)(x+3)$. Since the numerator is always nonzero, there is a vertical asymptote whenever the denominator is $0$, which occurs for $x = 2$ and $x = -3$.  Therefore, the graph has $\\boxed{2}$ vertical asymptotes.'},
 {'problem': 'What is the positive difference between $120\\%$ of 30 and $130\\%$ of 20?',
  'level': 'Level 1',
  'type': 'Algebra',
  'solution': 'One hundred twenty percent of 30 is $120\\cdot30\\cdot\\frac{1}{100}=36$, and $130\\%$ of 20 is $ 130\\cdot 20\\cdot\\frac{1}{100}=26$.  The difference between 36 and 26 is $\\boxed{10}$.'},
 {'problem': 'Find $x$ such that $\\lceil x \\rceil + x = \\dfrac{23}{7}$. Express $x$ as a common fraction.',
  'level': 'Level 4',
  'type': 'Algebra',
  'solution': 'First, we note that $x$ must be positive, since otherwise $\\

In [13]:
import spacy

# Load the English model
nlp = spacy.load("en_core_web_lg")


In [22]:
def convert_to_list_of_sentences(text):
    sents = text.split(". ")
    sents = [s+'.' for s in sents[:-1]] + [sents[-1]]
    return sents

train_ds = []
for i, ex in enumerate(ds['train']):
    
    l = convert_to_list_of_sentences(ex['solution'])
    solution = "\n".join(l)

    sample = {
        'problem': ex['problem'],
        'solution': solution,
        'level': ex['level']
    }
    print(solution)
    print("*******************")
    
    if i > 10:
        break
    

For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$.
For example, $ax+3$ and $x-5$ must be equal when $x=2$.
This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \Rightarrow a=-3$.
Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$.
Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$.
So $a+b=-3+3=\boxed{0}$.
*******************
Let $x$ be the number of band members in each row for the original formation, when two are left over.
 Then we can write two equations from the given information: $$rx+2=m$$ $$(r-2)(x+1)=m$$ Setting these equal, we find: $$rx+2=(r-2)(x+1)=rx-2x+r-2$$ $$2=-2x+r-2$$ $$4=r-2x$$ We know that the band has less than 100 members.
 Based on the first equation, we must have $rx$ less than 98.
 We can guess and check some values of $r$ and $x$ in the last equation.
 If $r=18$, then $x=7$, and $rx=126$ which is too big.
 If $r=16$, then $x=6$, and $rx=96$, which is less than 98.
 Checking back in the second formation, we see that 

# Buffer